# Dynamic Programming
---
> 동적계획법 실습
1. Policy Iteration
2. Value Iteration

- D.P 는 공통적으로 Model-based 방법이다
* 둘 다 백업 다이어그램과 수식을 함께 고려하면 확실한 이해를 할 수 있음.
* 기댓값을 계산, 평균과 최댓값을 구하는 과정 모두 제대로 이해해야 함
* max 가 하나 들어갈 때 마다 기댓값이 하나 빠짐
* q 가 v 가 되면 혹은 역으로 되어도 기댓값이 하나 빠짐

### 1. Policy Iteration
---
>Policy Evaluation (평가)<br>
Policy Improvement (발전)

- On - Policy method
- 크게 '평가 발전' 2 가지 과정으로 이루어진다.
- 평가 한 번 하고 발전 한 번 하는 것을 GPI (Generalized Policy Iteration) 이라고 한다.
- Bellman Expected Equation 을 사용한다.
- equation 을 iteration 이 가능한 점화식으로 변환하여 알고리즘을 만듬
- Value 와 Policy 가 구분되어 있고 따로따로 업데이트 한다.
- 엄연히 value 와 policy 가 분리되어 있음.
    * policy 는 q 가 아니라 확률분포인 pi 임!
- 업데이트 식은 기댓값 계산이므로 sigma 합의 형태임

In [3]:
import numpy as np

In [20]:
class Policy_Iteration:
    
    def __init__(self):
        self.n_map = 4
        self.n_action = 4
        self.gamma = 0.9
        self.grid_world = np.array(
                    [0,0,0,0,
                     0,0,0,0,
                     0,0,0,1,
                     1,0,0,1]).reshape(self.n_map,-1)
        self.value = np.zeros_like(self.grid_world)
        self.policy = np.ones([self.n_action, self.n_map, self.n_map]) / self.n_action
    
    
    '''
    Evaluation
    '''
    def Evaluation(self):
        v_tmp = np.zeros_like(self.grid_world, dtype=np.float16)
        for row in range(self.n_map):
            for col in range(self.n_map):
                for a in range(self.n_action):
                    reward = self.get_reward(a, row, col)
                    next_state = self.get_state(a, row, col)

                    # P.I 확률계산식 - bellman expected equation. 평균값 구하는 과정
                    v_tmp[row, col] += self.policy[a, row, col] * (reward + self.gamma * self.value[next_state[0], next_state[1]])
        
        v_tmp[3,3] = 0
        self.value = v_tmp
    
    
    '''
    Imporvement
    '''
    def Improvement(self):
        for row in range(self.n_map):
            for col in range(self.n_map):
                next_p = np.zeros([4], dtype=np.float16)
                cnt = 1
                max_tmp = -9999
                idx = []
                
                for a in range(self.n_action):
                    next_state = self.get_state(a, row, col)
                    # 최댓값 구하는 과정. Q 를 토대로 V 보다 큰 값이면서 최대를 고르게 하는게 공식인데, 코드에선 그냥 함축되어 있어 주의 필요
                    # greedy 하게 업데이트 하므로 결국 최대 q 를 고름
                    tmp = self.get_reward(a, row, col) + self.gamma * self.value[next_state[0], next_state[1]]
                    
                    if(tmp > max_tmp):
                        max_tmp = tmp
                        idx.clear()
                        cnt = 1
                        idx.append(a)
                        
                    elif(tmp == max_tmp):
                        cnt += 1
                        idx.append(a)
                        
                prob = 1.0 / cnt
                
                for i in idx:
                    next_p[i] = prob
                    
                self.policy[ : , row, col] = next_p
    
    
    '''
    R(s,a)
    P(s'|s,a)
    
    0  1  2  3
    상 하 좌 우
    '''
    def get_reward(self, a, r, c):
        
        if(a == 0 and r > 0):
            r -= 1

        elif(a == 1 and r < 3):
            r += 1

        elif(a == 2 and c > 0):
            c -= 1

        elif(a == 3 and c < 3):
            c += 1

        # 맵 넘어가면 부적 보상
        else:
            return -100

        reward = self.grid_world[r, c]

        return reward

    
    '''
    next state
    '''
    def get_state(self, a, r, c):
        
        if(a == 0 and r > 0):
            r -= 1

        elif(a == 1 and r < 3):
            r += 1

        elif(a == 2 and c > 0):
            c -= 1

        elif(a == 3 and c < 3):
            c += 1
            
        return r, c
    
    def get_policy(self):
        policy = np.zeros_like(self.grid_world)
        
        for row in range(self.n_map):
            for col in range(self.n_map):
                policy[row, col] = np.argmax(self.policy[ : , row, col])
                
        return policy

In [21]:
p_i = Policy_Iteration()

In [22]:
for i in range(10):
    # GPI(Generalized Policy Improvement) - 1 Eval --> 1 Improve
    p_i.Evaluation()
    p_i.Improvement()

print(p_i.grid_world)
print(p_i.value)
print(p_i.policy)
print(p_i.get_policy())

[[0 0 0 0]
 [0 0 0 0]
 [0 0 0 1]
 [1 0 0 1]]
[[1.999 2.227 2.475 3.127]
 [2.648 2.475 3.127 3.475]
 [2.998 3.127 3.475 3.127]
 [2.648 2.998 3.127 0.   ]]
[[[0.  0.  0.  0. ]
  [0.  0.  0.  0. ]
  [0.  0.  0.  0.5]
  [0.5 0.  1.  1. ]]

 [[1.  0.5 0.5 1. ]
  [1.  0.5 0.5 1. ]
  [1.  0.  0.  0. ]
  [0.  0.  0.  0. ]]

 [[0.  0.  0.  0. ]
  [0.  0.  0.  0. ]
  [0.  0.  0.  0.5]
  [0.  1.  0.  0. ]]

 [[0.  0.5 0.5 0. ]
  [0.  0.5 0.5 0. ]
  [0.  1.  1.  0. ]
  [0.5 0.  0.  0. ]]]
[[1 1 1 1]
 [1 1 1 1]
 [1 3 3 0]
 [0 2 0 0]]
